In [ ]:
#import necessary packages
import os
import sys
import nltk
import gensim
import re
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from gensim import corpora
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm




In [ ]:
# Tokenizer 
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = stopwords.words("english")


# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
s_stemmer = SnowballStemmer("english")

# Create listFileName for all the files in diabetes corpus
listFileName = [fileName for fileName in os.listdir(os.getcwd()+"/diabetes/") if ".txt" in str(fileName)]



In [ ]:
# Build topic modeling for individual documents in the corpus
texts = []
topicsAppend = []
globalWordRaw = []
globalWordList = []
pertopic = []
listwords = []
index = 0
for file in listFileName:    
    filepath = open(os.getcwd()+"/diabetes/"+file)
    i = filepath.read()
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]    
    # stem tokens
    stemmed_tokens = [s_stemmer.stem(i) for i in stopped_tokens]    
    # add tokens to list
    texts.append(stemmed_tokens)
    filepath.close()
    # start topic modelling process
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=1, id2word = dictionary, passes=1)
    topicsAppend.append(ldamodel.print_topics(num_topics=1, num_words=30))
    tmplistwords = re.findall(r'\"[\w+]+\"',topicsAppend[index][0][1])
    index = index + 1
    for x in listwords:
        pertopic.append(x)
    globalWordRaw.append(" ".join(pertopic))
    listwords = [i.replace('"', '') for i in tmplistwords]
    globalWordList.append(listwords)
    texts = []
    pertopic = []
    listwords = []




In [ ]:
# building the corpus topic modelling 
texts = []
for file in listFileName:    
    filepath = open(os.getcwd()+"/diabetes/"+file)
    i = filepath.read()
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop]  
    stemmed_tokens = [s_stemmer.stem(i) for i in stopped_tokens]
    texts.append(stemmed_tokens)
    filepath.close()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=1, id2word = dictionary, passes=1)
tmpglobalTopic = re.findall(r'\"[\w+]+\"',ldamodel.print_topics(num_topics=1, num_words=30)[0][1])
globalTopic = [i.replace('"', '') for i in tmpglobalTopic]
print(ldamodel.print_topics(num_topics=1, num_words=30))

#Build feature set for our model
globalVector = []
globalTopicVector = []
for g2 in globalWordList:
    common = set(globalTopic).intersection(g2)
    common_new = list(common)
    tmpVector = [] 
    for x in globalTopic:
        try: 
            common_new.index(x) 
            tmpVector.append(1) 
        except: 
            tmpVector.append(0)
    globalVector.append(tmpVector)
    globalTopicVector.append(len(common_new))
    tmpVector = []


numofDiabetes = []
numofShockingWords = []
shockSum = 0
shockingWords = ["surpris", "astonish", "howev", "unexpect", "amaz", "shock", "astound"]
for myFile in listFileName:
    filepath = open(os.getcwd()+"/diabetes/"+myFile)
    i = filepath.read()
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop] 
    s_stemmed_tokens = [s_stemmer.stem(i) for i in stopped_tokens]
    fdist = FreqDist(s_stemmed_tokens)
    filepath.close()
    numofDiabetes.append(fdist["diabet"])
    for word in shockingWords:
        shockSum = shockSum + fdist[word]
    numofShockingWords.append(shockSum)
    shockSum = 0


#Build Model
mydf = pd.DataFrame(globalVector, columns=globalTopic, index=listFileName)
traindf = mydf

#Get label information from 2 users
labelsUser1df = pd.read_csv("fileName.csv", index_col="Filename")
labelsUser2df = pd.read_csv("supriseMe.csv", index_col="FileName")

#Train the model
trainclf = svm.SVC()
trainclf.fit(traindf[0:99], labelsUser2df["SupriseMe"][0:99])
trainforest = RandomForestClassifier(n_estimators = 100) 
trainforest = trainforest.fit( traindf[0:99], labelsUser2df["SupriseMe"][0:99] )

#Predict the model
resultforest_train = trainforest.predict(traindf[100:1000])
resultsvm_train = trainclf.predict(traindf[100:1000])

#Display the results of the suprising documents
resultdf = pd.DataFrame(index=traindf[100:1000].index.tolist())
resultdf = resultdf.assign(resultforest_train = resultforest_train)
resultdf = resultdf.assign(resultsvm_train = resultsvm_train)
pd.options.display.max_rows = 990
pd.options.display.max_columns = 40
resultdf[resultdf["resultforest_train"] == "1"]
resultdf[resultdf["resultsvm_train"] == "1"]